In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

Using TensorFlow backend.


In [2]:
df = pd.read_csv('train.csv')
y = df['label'].to_numpy()
X = df.drop('label', axis = 1)
X = X.to_numpy().reshape(df.shape[0],28,28,1)

df_extra = pd.read_csv('Dig-MNIST.csv')
y_extra = df_extra['label'].to_numpy()
X_extra = df_extra.drop('label', axis = 1)
X_extra = X_extra.to_numpy().reshape(df_extra.shape[0],28,28,1)


X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.10, random_state=777)



df_test = pd.read_csv('test.csv')
X_test = df_test.drop('id', axis=1).to_numpy().reshape(df_test.shape[0],28,28,1)

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255.,
                                   rotation_range=10,
                                   width_shift_range=0.25,
                                   height_shift_range=0.25,
                                   shear_range=0.1,
                                   zoom_range=0.25,
                                   horizontal_flip=False)

valid_datagen = ImageDataGenerator(rescale=1./255.)

In [4]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, kernel_size=3, activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, kernel_size=3, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, kernel_size=5, padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Conv2D(128, kernel_size=3, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(128, kernel_size=3, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(128, kernel_size=5, padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Conv2D(256, kernel_size=3, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
batch_normalization (BatchNo (None, 26, 26, 64)        256       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 24, 24, 64)        256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        102464    
_________________________________________________________________
batch_normalization_2 (Batch (None, 24, 24, 64)        256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0

In [6]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                            patience=200,
                                            verbose=1,
                                            factor=0.2)

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)

In [7]:
history = model.fit_generator(train_datagen.flow(X_train, y_train, batch_size=1024),
                              steps_per_epoch=100,
                              epochs=50,
                              validation_data=valid_datagen.flow(X_val, y_val),
                              validation_steps=50,
                              callbacks=[learning_rate_reduction, es])

Epoch 1/50
100/100 [==============================] - 55s 554ms/step - loss: 0.5525 - accuracy: 0.8132 - val_loss: 3.0949 - val_accuracy: 0.1194
Epoch 2/50
100/100 [==============================] - 41s 408ms/step - loss: 0.1229 - accuracy: 0.9592 - val_loss: 10.7546 - val_accuracy: 0.1044
Epoch 3/50
100/100 [==============================] - 41s 412ms/step - loss: 0.0886 - accuracy: 0.9708 - val_loss: 14.0671 - val_accuracy: 0.1044
Epoch 4/50
100/100 [==============================] - 41s 410ms/step - loss: 0.0672 - accuracy: 0.9777 - val_loss: 12.9768 - val_accuracy: 0.1044
Epoch 5/50
100/100 [==============================] - 41s 412ms/step - loss: 0.0601 - accuracy: 0.9801 - val_loss: 11.3265 - val_accuracy: 0.1481
Epoch 6/50
100/100 [==============================] - 41s 410ms/step - loss: 0.0537 - accuracy: 0.9826 - val_loss: 3.2359 - val_accuracy: 0.4044
Epoch 7/50
100/100 [==============================] - 41s 408ms/step - loss: 0.0486 - accuracy: 0.9841 - val_loss: 0.2596 - va

In [10]:
y_pred = model.predict_classes(X_test * 1.0 / 255.0)

In [13]:
answer = pd.DataFrame(y_pred, columns=['label'])
answer.index.name = 'id'
answer.to_csv('submission.csv')